# Convert the DOT json output to my customized json format, which is ready for multi-level metrics

In [1]:
from shapely.geometry import *
import json

In [39]:
dotjson = json.load(open('../../../data/coauthor/sfdp.dot.json'))
output_path = '../../../data/coauthor/sfdp.json'

In [40]:
bbox = [float(x) for x in dotjson['bb'].split(',')]
bbox

[0.0, 0.0, 477.4, 464.62]

In [41]:
nodes = []
for node in dotjson['objects']:
    nid = int(node['name'])
    pos = [float(x) for x in node['pos'].split(',')]
    # print(nid, pos)
    d = float(node['height'])
    g = Point(pos[0], pos[1]).buffer(d / 2.0, cap_style=CAP_STYLE.round)
    nodes.append({'id': nid, 'parent_metanode': None, 'geometry': g, 'diameter': d})
# nodes
len(nodes)

1808

In [42]:
edges = []
for edge in dotjson['edges']:
    ends = (edge['tail'], edge['head'])
    eid = '{}-{}'.format(ends[0], ends[1])
    points = edge['_draw_'][-1]['points']
    s = ((points[0][0] + points[1][0]) / 2, (points[0][1] + points[1][1]) / 2)
    e = ((points[2][0] + points[3][0]) / 2, (points[2][1] + points[3][1]) / 2)
    g = LineString([s, e])
    edges.append({'id': eid, 'ends': ends, 'geometry': g})
    # print(eid, s, e)
# edges
len(edges)

9387

In [43]:
for n in nodes:
    n['geometry'] = mapping(n['geometry'])
for e in edges:
    e['geometry'] = mapping(e['geometry'])

json_data = {
    'leaf_nodes': nodes,
    'edges': edges,
    'height': 1,
    'root': None,
    'metanodes': {},
    'bounding_box': [[bbox[0], bbox[1]], [bbox[2], bbox[3]]]
}
json.dump(json_data, open(output_path, 'w'))